<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Изучение-и-подготовка-данных" data-toc-modified-id="Изучение-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение и подготовка данных</a></span></li><li><span><a href="#Анализ-различных-моделей-для-определения-стоимости-автомобиля" data-toc-modified-id="Анализ-различных-моделей-для-определения-стоимости-автомобиля-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Анализ различных моделей для определения стоимости автомобиля</a></span><ul class="toc-item"><li><span><a href="#Линейная-регрессия" data-toc-modified-id="Линейная-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Линейная регрессия</a></span></li><li><span><a href="#LightGBM" data-toc-modified-id="LightGBM-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LightGBM</a></span></li><li><span><a href="#XGBoost" data-toc-modified-id="XGBoost-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>XGBoost</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>CatBoost</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Описание данных
- Признаки
    - DateCrawled — дата скачивания анкеты из базы
    - VehicleType — тип автомобильного кузова
    - RegistrationYear — год регистрации автомобиля
    - Gearbox — тип коробки передач
    - Power — мощность (л. с.)
    - Model — модель автомобиля
    - Kilometer — пробег (км)
    - RegistrationMonth — месяц регистрации автомобиля
    - FuelType — тип топлива
    - Brand — марка автомобиля
    - NotRepaired — была машина в ремонте или нет
    - DateCreated — дата создания анкеты
    - NumberOfPictures — количество фотографий автомобиля
    - PostalCode — почтовый индекс владельца анкеты (пользователя)
    - LastSeen — дата последней активности пользователя
- Целевой признак
    - Price — цена (евро)

## Изучение и подготовка данных

In [1]:
import pandas as pd
import numpy as np

from tqdm.auto import tqdm
tqdm.pandas()

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [2]:
# Код для чтения данных
df_autos = pd.read_csv('df_autos.csv')  # Локальный путь

In [3]:
df_autos.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
df_autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

В данных много пропусков, причем разное количество. Следовательно пропуски не в одних и тех же строках, поэтому рассмотрим каждый столбец с пропусками!

In [5]:
# Найдем дубликаты
df_autos[df_autos.duplicated() == True]

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
171088,2016-03-08 18:42:48,1799,coupe,1999,auto,193,clk,20000,7,petrol,mercedes_benz,no,2016-03-08 00:00:00,0,89518,2016-03-09 09:46:57
231258,2016-03-28 00:56:10,1000,small,2002,manual,83,other,150000,1,petrol,suzuki,no,2016-03-28 00:00:00,0,66589,2016-03-28 08:46:21
258109,2016-04-03 09:01:15,4699,coupe,2003,auto,218,clk,125000,6,petrol,mercedes_benz,yes,2016-04-03 00:00:00,0,75196,2016-04-07 09:44:54
325651,2016-03-18 18:46:15,1999,wagon,2001,manual,131,passat,150000,7,gasoline,volkswagen,no,2016-03-18 00:00:00,0,36391,2016-03-18 18:46:15


In [6]:
# Удалим дубликаты
df_autos = df_autos[df_autos.duplicated() != True]
df_autos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354365 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354365 non-null  object
 1   Price              354365 non-null  int64 
 2   VehicleType        316875 non-null  object
 3   RegistrationYear   354365 non-null  int64 
 4   Gearbox            334532 non-null  object
 5   Power              354365 non-null  int64 
 6   Model              334660 non-null  object
 7   Kilometer          354365 non-null  int64 
 8   RegistrationMonth  354365 non-null  int64 
 9   FuelType           321470 non-null  object
 10  Brand              354365 non-null  object
 11  NotRepaired        283211 non-null  object
 12  DateCreated        354365 non-null  object
 13  NumberOfPictures   354365 non-null  int64 
 14  PostalCode         354365 non-null  int64 
 15  LastSeen           354365 non-null  object
dtypes: int64(7), object(

Заполним пропуски в колонке с моделью автомобиля, для этого изучим уникальные значения моделей.

In [7]:
df_autos['Model'].unique()

array(['golf', nan, 'grand', 'fabia', '3er', '2_reihe', 'other', 'c_max',
       '3_reihe', 'passat', 'navara', 'ka', 'polo', 'twingo', 'a_klasse',
       'scirocco', '5er', 'meriva', 'arosa', 'c4', 'civic', 'transporter',
       'punto', 'e_klasse', 'clio', 'kadett', 'kangoo', 'corsa', 'one',
       'fortwo', '1er', 'b_klasse', 'signum', 'astra', 'a8', 'jetta',
       'fiesta', 'c_klasse', 'micra', 'vito', 'sprinter', '156', 'escort',
       'forester', 'xc_reihe', 'scenic', 'a4', 'a1', 'insignia', 'combo',
       'focus', 'tt', 'a6', 'jazz', 'omega', 'slk', '7er', '80', '147',
       '100', 'z_reihe', 'sportage', 'sorento', 'v40', 'ibiza', 'mustang',
       'eos', 'touran', 'getz', 'a3', 'almera', 'megane', 'lupo', 'r19',
       'zafira', 'caddy', 'mondeo', 'cordoba', 'colt', 'impreza',
       'vectra', 'berlingo', 'tiguan', 'i_reihe', 'espace', 'sharan',
       '6_reihe', 'panda', 'up', 'seicento', 'ceed', '5_reihe', 'yeti',
       'octavia', 'mii', 'rx_reihe', '6er', 'modus', 'fox'

In [8]:
# Вывод строк с моделью 'other'
df_autos.query('Model == "other"')

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
24,2016-03-13 20:40:49,500,sedan,1990,manual,118,other,150000,10,petrol,mercedes_benz,yes,2016-03-13 00:00:00,0,35390,2016-03-13 20:40:49
38,2016-04-01 17:45:07,11900,other,2002,manual,129,other,150000,11,gasoline,volkswagen,no,2016-04-01 00:00:00,0,10551,2016-04-05 12:47:30
39,2016-03-25 15:50:30,1500,bus,1984,manual,70,other,150000,8,gasoline,mercedes_benz,no,2016-03-25 00:00:00,0,22767,2016-03-27 03:17:02
52,2016-04-04 10:57:36,1400,NaN,2016,manual,55,other,5000,1,NaN,hyundai,NaN,2016-04-04 00:00:00,0,34454,2016-04-06 12:45:43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354290,2016-03-11 14:45:44,2500,small,2007,manual,60,other,125000,3,petrol,hyundai,NaN,2016-03-11 00:00:00,0,18107,2016-03-17 21:46:52
354302,2016-03-12 08:37:15,5990,bus,1984,auto,0,other,70000,4,petrol,chevrolet,NaN,2016-03-12 00:00:00,0,87600,2016-03-24 23:47:48
354316,2016-03-07 19:58:44,3300,coupe,1957,manual,40,other,100000,11,petrol,trabant,no,2016-03-07 00:00:00,0,10317,2016-03-08 06:45:48
354325,2016-03-12 16:48:14,12800,convertible,1991,manual,211,other,150000,7,petrol,porsche,no,2016-03-12 00:00:00,0,21147,2016-04-07 06:46:00


Среди всех моделей есть значение 'other' и оно у разных марок автомобилей, значит все пропуски можно заменить на это значение!

In [9]:
df_autos['Model'] = df_autos['Model'].fillna(value='other')
df_autos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354365 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354365 non-null  object
 1   Price              354365 non-null  int64 
 2   VehicleType        316875 non-null  object
 3   RegistrationYear   354365 non-null  int64 
 4   Gearbox            334532 non-null  object
 5   Power              354365 non-null  int64 
 6   Model              354365 non-null  object
 7   Kilometer          354365 non-null  int64 
 8   RegistrationMonth  354365 non-null  int64 
 9   FuelType           321470 non-null  object
 10  Brand              354365 non-null  object
 11  NotRepaired        283211 non-null  object
 12  DateCreated        354365 non-null  object
 13  NumberOfPictures   354365 non-null  int64 
 14  PostalCode         354365 non-null  int64 
 15  LastSeen           354365 non-null  object
dtypes: int64(7), object(

Заполним пропуски в колонке с коробкой передач автомобиля, для этого изучим уникальные значения.

In [10]:
df_autos['Gearbox'].unique()

array(['manual', 'auto', nan], dtype=object)

Все пропуски заменим на значение 'indefinite' (неопределенный)

In [11]:
df_autos['Gearbox'] = df_autos['Gearbox'].fillna(value='indefinite')

Заполним пропуски в колонке с типом топлива автомобиля, для этого изучим уникальные значения.

In [12]:
df_autos['FuelType'].unique()

array(['petrol', 'gasoline', nan, 'lpg', 'other', 'hybrid', 'cng',
       'electric'], dtype=object)

Среди значений так же есть 'other', пропуски заменим на это значение.

In [13]:
df_autos['FuelType'] = df_autos['FuelType'].fillna(value='other')

Заполним пропуски в колонке с информацией о ремонте автомобиля, для этого изучим уникальные значения.

In [14]:
df_autos['NotRepaired'].unique()

array([nan, 'yes', 'no'], dtype=object)

In [15]:
df_autos['NotRepaired'].value_counts()

no     247158
yes     36053
Name: NotRepaired, dtype: int64

Будем считать пропуск отсутствием ремонта, поэтому заменим 'nan' на 'no'

In [16]:
df_autos['NotRepaired'] = df_autos['NotRepaired'].fillna(value='no')

Заполним пропуски в колонке с кузовом автомобиля, для этого изучим уникальные значения.

In [17]:
df_autos['VehicleType'].unique()

array([nan, 'coupe', 'suv', 'small', 'sedan', 'convertible', 'bus',
       'wagon', 'other'], dtype=object)

In [18]:
len(df_autos['Model'].unique())

250

In [19]:
len(df_autos['Brand'].unique())

40

Напишем функцию, которая:
- находит все строки с требуемыми маркой и моделью авто
- возвращает самый встречаемый тип кузова среди этих строк

In [20]:
def model_brands_pairs(df_autos=df_autos):
    # Создадим новую таблицу, куда будет записываться модели, марки и наиболее часто встречаемый тип кузова
    df_test = pd.DataFrame(columns=['model', 'brand', 'vehicle_type'])
    
    min_index = 0

    for model in tqdm(df_autos['Model'].unique()):
        for brand in df_autos['Brand'].unique():
            # Найдем все строки с требуемыми маркой и моделью авто
            df_temp = df_autos[(df_autos['Brand'] == brand) & (df_autos['Model'] == model)]
            # Выберем ненулевой длины срезы
            if len(df_temp) == 0:
                continue
            # найдем наиболее часто встречаемый тип кузова при таком сочетании марки и модели
            vehicle_type = df_temp['VehicleType'].value_counts().index[0]
            # в минимальный индекс записываем модель, бренд, тип кузова
            df_test.loc[min_index, 'model'] = model
            df_test.loc[min_index, 'brand'] = brand
            df_test.loc[min_index, 'vehicle_type'] = vehicle_type
            # обновляем индекс
            min_index += 1
    return df_test

In [21]:
try:
    # Откроем таблицу сочетаний модели, марки и  типа кузова
    df_test = pd.read_csv('pair_model_brand.csv')
except FileNotFoundError:
    # если таблицы нет, сгенерируем ее и сохраним
    df_test = model_brands_pairs()
    df_test.to_csv('pair_model_brand.csv', index=False)

  0%|          | 0/250 [00:00<?, ?it/s]

In [22]:
df_test.head()

,model,brand,vehicle_type
0,golf,volkswagen,sedan
1,other,volkswagen,small
2,other,audi,wagon
3,other,jeep,suv
4,other,skoda,small


In [23]:
# Заменим пропуски в типе кузова в исходной таблице на наиболее встречаемый при
# определенном сочетании марки и модели
def replace_vehicle_nan(df_autos=df_autos):
    for index in tqdm(df_autos[df_autos['VehicleType'].isna()].index):
        model = df_autos.loc[index, 'Model']
        brand = df_autos.loc[index, 'Brand']
        vehicle_type = df_test[(df_test['brand'] == brand)
                               &(df_test['model'] == model)]['vehicle_type'].values[0]
        df_autos.loc[index, 'VehicleType'] = vehicle_type
    return df_autos

In [24]:
try:
    # Откроем таблицу сочетаний модели, марки и  типа кузова
    df_autos_new = pd.read_csv('df_autos_new.csv')
except FileNotFoundError:
    # если таблицы нет, сгенерируем ее и сохраним
    df_autos_new = replace_vehicle_nan()
    df_autos_new.to_csv('df_autos_new.csv', index=False)

  0%|          | 0/37490 [00:00<?, ?it/s]

In [25]:
# Найдем минимальную стоимость
df_autos_new['Price'].min()

0

Минимальная стоимость равна нулю, это ошибка, которая внесет в обучение модели неточности, посмотрим какое количество объявлений с нулевой стоимостью

In [26]:
len(df_autos_new.query('Price == 0'))

10772

In [27]:
df_autos_new = df_autos_new.query('Price != 0')

In [28]:
# Найдем количесвто объявлений с единичной стоимостью!
len(df_autos_new.query('Price == 1'))

1189

In [29]:
# Таких объявлений мало? поэтому удалим их
df_autos_new = df_autos_new.query('Price != 1')

In [30]:
df_autos_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 342404 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        342404 non-null  object
 1   Price              342404 non-null  int64 
 2   VehicleType        342404 non-null  object
 3   RegistrationYear   342404 non-null  int64 
 4   Gearbox            342404 non-null  object
 5   Power              342404 non-null  int64 
 6   Model              342404 non-null  object
 7   Kilometer          342404 non-null  int64 
 8   RegistrationMonth  342404 non-null  int64 
 9   FuelType           342404 non-null  object
 10  Brand              342404 non-null  object
 11  NotRepaired        342404 non-null  object
 12  DateCreated        342404 non-null  object
 13  NumberOfPictures   342404 non-null  int64 
 14  PostalCode         342404 non-null  int64 
 15  LastSeen           342404 non-null  object
dtypes: int64(7), object(

In [31]:
# Удалим колонки с датой скачивания анкеты, датой создания анкеты и датой последней сессии
df_autos_new = df_autos_new.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis=1)
df_autos_new.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,NumberOfPictures,PostalCode
0,480,sedan,1993,manual,0,golf,150000,0,petrol,volkswagen,no,0,70435
1,18300,coupe,2011,manual,190,other,125000,5,gasoline,audi,yes,0,66954
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,no,0,90480
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,0,91074
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,0,60437


In [32]:
df_autos_new = df_autos_new.drop(['RegistrationMonth'], axis=1)
df_autos_new.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,NumberOfPictures,PostalCode
0,480,sedan,1993,manual,0,golf,150000,petrol,volkswagen,no,0,70435
1,18300,coupe,2011,manual,190,other,125000,gasoline,audi,yes,0,66954
2,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,no,0,90480
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no,0,91074
4,3600,small,2008,manual,69,fabia,90000,gasoline,skoda,no,0,60437


Ошибки обработаны, пропуски заполнены, можно переходить к обучению моделей!

In [33]:
# Найдем машины старше 1900 и старше 2021, очевидно, что это ошибки
df_autos_new.query('RegistrationYear < 1900 or RegistrationYear > 2021')

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,NumberOfPictures,PostalCode
12946,49,sedan,5000,indefinite,0,golf,5000,other,volkswagen,no,0,74523
15870,1700,sedan,3200,indefinite,0,other,5000,other,sonstige_autos,no,0,33649
16062,190,wagon,1000,indefinite,0,mondeo,5000,other,ford,no,0,47166
17271,700,small,9999,indefinite,0,other,10000,other,opel,no,0,21769
17346,6500,sedan,8888,indefinite,0,other,10000,other,sonstige_autos,no,0,55262
...,...,...,...,...,...,...,...,...,...,...,...,...
331753,99,sedan,9999,indefinite,0,1er,10000,other,bmw,no,0,1239
334967,12000,sedan,4000,indefinite,500,golf,5000,other,volkswagen,no,0,57392
338829,50,sedan,3000,indefinite,3000,golf,100000,other,volkswagen,yes,0,23992
340759,700,sedan,1600,manual,1600,a3,150000,petrol,audi,no,0,86343


In [34]:
# Удалим эти значения
df_autos_new = df_autos_new.query('1900 <= RegistrationYear <= 2021')

In [35]:
# Найдем все авто, у которых мощность больше 500
df_autos_new.query('Power > 500')

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,NumberOfPictures,PostalCode
1816,3200,small,2004,manual,1398,corolla,5000,petrol,toyota,no,0,22043
4060,3100,sedan,2005,manual,953,colt,150000,gasoline,mitsubishi,no,0,60326
5328,500,wagon,1999,manual,1001,astra,150000,petrol,opel,no,0,33154
6296,599,small,2002,manual,603,matiz,5000,petrol,chevrolet,yes,0,44379
6504,3000,small,2009,manual,771,punto,125000,petrol,fiat,no,0,40721
...,...,...,...,...,...,...,...,...,...,...,...,...
348968,250,small,1999,manual,1241,ypsilon,150000,petrol,lancia,yes,0,28259
351947,1500,bus,2001,manual,1001,zafira,5000,gasoline,opel,no,0,66117
353493,12500,sedan,2017,manual,2000,other,60000,gasoline,chrysler,no,0,44145
353633,2400,sedan,2007,manual,650,c2,150000,petrol,citroen,no,0,45277


In [36]:
# Всего 417 значений, в основном это ошибки, но ничего страшного если удалим несколько машин
# с реальной такой мощностью
df_autos_new = df_autos_new.query('Power <= 500')

In [37]:
# Найдем количество автомобилей с нулевой мощностью
len(df_autos_new.query('Power == 0'))

35778

In [38]:
df_autos_new.head(10)

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,NumberOfPictures,PostalCode
0,480,sedan,1993,manual,0,golf,150000,petrol,volkswagen,no,0,70435
1,18300,coupe,2011,manual,190,other,125000,gasoline,audi,yes,0,66954
2,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,no,0,90480
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no,0,91074
4,3600,small,2008,manual,69,fabia,90000,gasoline,skoda,no,0,60437
5,650,sedan,1995,manual,102,3er,150000,petrol,bmw,yes,0,33775
6,2200,convertible,2004,manual,109,2_reihe,150000,petrol,peugeot,no,0,67112
8,14500,bus,2014,manual,125,c_max,30000,petrol,ford,no,0,94505
9,999,small,1998,manual,101,golf,150000,other,volkswagen,no,0,27472
10,2000,sedan,2004,manual,105,3_reihe,150000,petrol,mazda,no,0,96224


In [39]:
def replace_power_zero(df_autos_new=df_autos_new):
    for row in tqdm(df_autos_new.index):
        if df_autos_new['Power'][row] == 0:
            df_autos_new['Power'][row] = df_autos_new[(df_autos_new['Model'] == df_autos_new['Model'][row])
                                                      & (df_autos_new['Power'] != 0 )]['Power'].mean()
        else:
            continue
    return df_autos_new

In [40]:
try:
    # Откроем таблицу сочетаний модели, марки и  типа кузова
    df_autos_new_01 = pd.read_csv('df_autos_new_01.csv')
except FileNotFoundError:
    # если таблицы нет, сгенерируем ее и сохраним
    df_autos_new_01 = replace_power_zero()
    df_autos_new_01.to_csv('df_autos_new_01.csv', index=False)

  0%|          | 0/341856 [00:00<?, ?it/s]

/opt/anaconda3/envs/practicum/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/anaconda3/envs/practicum/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [41]:
df_autos_new_01.head(10)

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,NumberOfPictures,PostalCode
0,480,sedan,1993,manual,104.0,golf,150000,petrol,volkswagen,no,0,70435
1,18300,coupe,2011,manual,190.0,other,125000,gasoline,audi,yes,0,66954
2,9800,suv,2004,auto,163.0,grand,125000,gasoline,jeep,no,0,90480
3,1500,small,2001,manual,75.0,golf,150000,petrol,volkswagen,no,0,91074
4,3600,small,2008,manual,69.0,fabia,90000,gasoline,skoda,no,0,60437
5,650,sedan,1995,manual,102.0,3er,150000,petrol,bmw,yes,0,33775
6,2200,convertible,2004,manual,109.0,2_reihe,150000,petrol,peugeot,no,0,67112
8,14500,bus,2014,manual,125.0,c_max,30000,petrol,ford,no,0,94505
9,999,small,1998,manual,101.0,golf,150000,other,volkswagen,no,0,27472
10,2000,sedan,2004,manual,105.0,3_reihe,150000,petrol,mazda,no,0,96224


In [42]:
df_autos_new_01['Power'] = df_autos_new_01['Power'].round()
df_autos_new_01.dropna(subset = ['Power'], inplace = True)
df_autos_new_01.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 341855 entries, 0 to 354368
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Price             341855 non-null  int64  
 1   VehicleType       341855 non-null  object 
 2   RegistrationYear  341855 non-null  int64  
 3   Gearbox           341855 non-null  object 
 4   Power             341855 non-null  float64
 5   Model             341855 non-null  object 
 6   Kilometer         341855 non-null  int64  
 7   FuelType          341855 non-null  object 
 8   Brand             341855 non-null  object 
 9   NotRepaired       341855 non-null  object 
 10  NumberOfPictures  341855 non-null  int64  
 11  PostalCode        341855 non-null  int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 33.9+ MB


In [43]:
df_autos_new_01['Power'] = df_autos_new_01['Power'].astype('int')

## Анализ различных моделей для определения стоимости автомобиля

In [44]:
# Выполним прямое кодирование таблицы с обходом дамми-ловушки
df_autos_new_ohe = pd.get_dummies(df_autos_new_01, drop_first=True)
df_autos_new_ohe

,Price,RegistrationYear,Power,Kilometer,NumberOfPictures,PostalCode,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,...,Brand_skoda,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_yes
0,480,1993,104,150000,0,70435,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,18300,2011,190,125000,0,66954,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,9800,2004,163,125000,0,90480,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1500,2001,75,150000,0,91074,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,3600,2008,69,90000,0,60437,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354363,1150,2000,125,150000,0,26624,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
354365,2200,2005,125,20000,0,39576,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
354366,1199,2000,101,125000,0,26135,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
354367,9200,1996,102,150000,0,87439,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [45]:
# Выделим переменные признаки и признак, который нужно предсказать для каждой таблицы
features = df_autos_new_ohe.drop('Price', axis=1)
target = df_autos_new_ohe['Price']
# Отделим валидационную выборку (25%) от первоначальных данных
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

### Линейная регрессия

In [46]:
start_time = datetime.now()

model_lin_reg = LinearRegression()
result_lin_reg = cross_val_score(model_lin_reg, features_train, target_train,
                                 scoring='neg_mean_squared_error', cv=5).mean()
end_time = datetime.now()
time_lin_reg = end_time - start_time
print('RMSE модели линейной регрессии:', round(abs(result_lin_reg)**0.5, 2))
print('Время выполнения кода:', time_lin_reg)

RMSE модели линейной регрессии: 2752.63
Время выполнения кода: 0:00:10.700842


In [47]:
# Найдем время обучения и предсказания модели линейной регрессии
start_time = datetime.now()
model_lin_reg.fit(features_train, target_train)
end_time = datetime.now()
time_lr_fit = end_time - start_time

start_time = datetime.now()
predictions=model_lin_reg.predict(features_test)
end_time = datetime.now()
time_lr_predict = end_time - start_time
print('Время обучения модели линейной регрессии:', time_lr_fit)
print('Время предсказания модели линейной регрессии:', time_lr_predict)

Время обучения модели линейной регрессии: 0:00:02.602821
Время предсказания модели линейной регрессии: 0:00:00.138819


### LightGBM

In [48]:
df_autos_new_01.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 341855 entries, 0 to 354368
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             341855 non-null  int64 
 1   VehicleType       341855 non-null  object
 2   RegistrationYear  341855 non-null  int64 
 3   Gearbox           341855 non-null  object
 4   Power             341855 non-null  int64 
 5   Model             341855 non-null  object
 6   Kilometer         341855 non-null  int64 
 7   FuelType          341855 non-null  object
 8   Brand             341855 non-null  object
 9   NotRepaired       341855 non-null  object
 10  NumberOfPictures  341855 non-null  int64 
 11  PostalCode        341855 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 33.9+ MB


In [49]:
# Поменяем тип данных на категориальный в следующих стобцах
for col in ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']:
    df_autos_new_01[col] = df_autos_new_01[col].astype('category')

In [50]:
df_autos_new_01.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 341855 entries, 0 to 354368
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   Price             341855 non-null  int64   
 1   VehicleType       341855 non-null  category
 2   RegistrationYear  341855 non-null  int64   
 3   Gearbox           341855 non-null  category
 4   Power             341855 non-null  int64   
 5   Model             341855 non-null  category
 6   Kilometer         341855 non-null  int64   
 7   FuelType          341855 non-null  category
 8   Brand             341855 non-null  category
 9   NotRepaired       341855 non-null  category
 10  NumberOfPictures  341855 non-null  int64   
 11  PostalCode        341855 non-null  int64   
dtypes: category(6), int64(6)
memory usage: 20.6 MB


In [51]:
# Выделим переменные признаки и признак, который нужно предсказать для каждой таблицы
features_categorial = df_autos_new_01.drop('Price', axis=1)
target_categorial = df_autos_new_01['Price']
# Отделим валидационную выборку (20%) от первоначальных данных
features_train_categorial, features_test_categorial, target_train_categorial, target_test_categorial = train_test_split(
    features_categorial, target_categorial, test_size=0.20, random_state=12345)

In [52]:
start_time = datetime.now()

model_lgbm = LGBMRegressor(random_state=12345)
params_lgbm = {'n_estimators' : [100, 200, 500],
               'max_depth' : range(2, 7),
               'num_leaves': [20, 30, 40],
               'learning_rate': [0.03, 0.1, 0.5]}

search_lgbm = GridSearchCV(estimator=model_lgbm, param_grid=params_lgbm, scoring='neg_mean_squared_error',
                           verbose=0, cv=3)
result_lgbm = search_lgbm.fit(features_train_categorial , target_train_categorial)

end_time = datetime.now()
time_lgbm = end_time - start_time

In [53]:
print('Лучшее значение RMSE в LightGBM:', round(abs(result_lgbm.best_score_)**0.5, 2))
print('Лучшие параметры LightGBM:', result_lgbm.best_params_)
print('Время выполнения кода:', time_lgbm)

Лучшее значение RMSE в LightGBM: 1622.5
Лучшие параметры LightGBM: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 500, 'num_leaves': 40}
Время выполнения кода: 0:05:53.249809


Мы перебрали много параметров, каждый щаг считался достаточно быстро: 1-2 секунды, в общем выполнение кода заняло 10 минут, это хорошее время для такого количество шагов, RMSE = 1621, это почти в два раза лучше результата в линейной регрессии.

In [54]:
# Найдем время обучения и предсказания модели LightGBM
start_time = datetime.now()
best_model_lgbm = search_lgbm.best_estimator_
best_model_lgbm.fit(features_train_categorial, target_train_categorial)
end_time = datetime.now()
time_lgbm_fit = end_time - start_time

start_time = datetime.now()
predictions=best_model_lgbm.predict(features_test_categorial)
end_time = datetime.now()
time_lgbm_predict = end_time - start_time
print('Время обучения модели LightGBM:', time_lgbm_fit)
print('Время предсказания модели LightGBM:', time_lgbm_predict)

Время обучения модели LightGBM: 0:00:02.306457
Время предсказания модели LightGBM: 0:00:00.416727


### XGBoost

In [55]:
start_time = datetime.now()

model_xgb = XGBRegressor(random_state=12345)
params_xgb = {'eta' : [0.1, 0.3, 0.5],
               'max_depth' : [6]}

search_xgb = GridSearchCV(estimator=model_xgb, param_grid=params_xgb, scoring='neg_mean_squared_error',
                           verbose=0, cv=3)
result_xgb = search_xgb.fit(features_train , target_train)

end_time = datetime.now()
time_xgb = end_time - start_time

In [56]:
print('Лучшее значение RMSE в XGBoost:', round(abs(result_xgb.best_score_)**0.5, 2))
print('Лучшие параметры XGBoost:', result_xgb.best_params_)
print('Время выполнения кода:', time_xgb)

Лучшее значение RMSE в XGBoost: 1701.6
Лучшие параметры XGBoost: {'eta': 0.5, 'max_depth': 6}
Время выполнения кода: 0:09:36.179775


Сделали всего 9 шагов, и каждый шаг занял около 53 секунд, это намного дольше чем в LightGBM, однако RMSE = 1704, что близко к результату lightGBM, но все же чуть больше.

In [57]:
# Найдем время обучения и предсказания модели XGBoost
start_time = datetime.now()
best_model_xbg = search_xgb.best_estimator_
best_model_xbg.fit(features_train, target_train)
end_time = datetime.now()
time_xgb_fit = end_time - start_time

start_time = datetime.now()
predictions=best_model_xbg.predict(features_test)
end_time = datetime.now()
time_xgb_predict = end_time - start_time
print('Время обучения модели XGBoost:', time_xgb_fit)
print('Время предсказания модели XGBoost:', time_xgb_predict)

Время обучения модели XGBoost: 0:01:27.976258
Время предсказания модели XGBoost: 0:00:00.283644


### CatBoost

In [ ]:
start_time = datetime.now()

model_cat = CatBoostRegressor(random_seed=12345)
params_cat = {'iterations' : [100, 300, 500],
              'depth' : range(4, 8),
              'grow_policy' : ['SymmetricTree']}

search_cat = GridSearchCV(estimator=model_cat, param_grid=params_cat, scoring='neg_mean_squared_error',
                           verbose=0, cv=3);
result_cat = search_cat.fit(features_train, target_train);

end_time = datetime.now()
time_cat = end_time - start_time

In [59]:
print('Лучшее значение RMSE в CatBoost:', round(abs(result_cat.best_score_)**0.5, 2))
print('Лучшие параметры CatBoost:', result_cat.best_params_)
print('Время выполнения кода:', time_cat)

Лучшее значение RMSE в CatBoost: 1659.82
Лучшие параметры CatBoost: {'depth': 7, 'grow_policy': 'SymmetricTree', 'iterations': 500}
Время выполнения кода: 0:01:53.796440


lightGBM выполнила 405 операций за 6 минут, это примерно 68 операций в минуту, CatBoost выполнила 36 операций за 1 минуту 40 сенкунд, это примерно 20 операций в минуту. CatBoost заметно дольше, даже с параметром grow_policy = SymmetricTree. RMSE этих моделей при этом примерно равны.

In [ ]:
# Найдем время обучения и предсказания модели CatBoost
start_time = datetime.now()
best_model_cat = search_cat.best_estimator_;
best_model_cat.fit(features_train, target_train);
end_time = datetime.now()
time_cat_fit = end_time - start_time

start_time = datetime.now()
predictions=best_model_cat.predict(features_test)
end_time = datetime.now()
time_cat_predict = end_time - start_time

In [69]:
print('Время обучения модели CatBoost:', time_cat_fit)
print('Время предсказания модели CatBoost:', time_cat_predict)

Время обучения модели CatBoost: 0:00:06.166688
Время предсказания модели CatBoost: 0:00:00.026105


In [61]:
# Перенесем результаты в единую таблицу
data = [['fit', time_lr_fit, time_lgbm_fit, time_xgb_fit, time_cat_fit],
       ['predict', time_lr_predict, time_lgbm_predict, time_xgb_predict, time_cat_predict]]
df_time = pd.DataFrame(data = data, columns = ['process', 'lin_reg', 'LightGBM', 'XGBoost', 'CatBoost'])
df_time

,process,lin_reg,LightGBM,XGBoost,CatBoost
0,fit,0 days 00:00:02.602821,0 days 00:00:02.306457,0 days 00:01:27.976258,0 days 00:00:06.115221
1,predict,0 days 00:00:00.138819,0 days 00:00:00.416727,0 days 00:00:00.283644,0 days 00:00:00.032022


## Выводы

- В ходе работы мы изучили данные
- Удалили дубликаты в данных; заполнили пропуски в колонках с моделью, типом кузова, типом топлива, информацией о ремонте автомобиля
- Написали функцию для нахождения самого частого типа кузова при определенной марке и модели автомобиля, заполнили пропуски в типах кузова
- Обучили различные модели: линейную регрессию, LightGBM, XGBoost, CatBoost, а так же сравнили скорость обучения моделей
- На наших данных быстрее всего обучается модель LightGBM, и лучшее значение RMSE = 1634 принадлежит этой модели.